Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
# PUT YOUR CODE HERE
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


In [ ]:
#1.
data = pd.read_csv("/content/drive/MyDrive/ВиМ/Лаба 1/sales_train.csv")
sales = pd.DataFrame(data)
result = sales.groupby('item_id', as_index=False)['item_cnt_day'].sum()
result = result.sort_values(by='item_cnt_day', ascending=False)
result.head(10)

,item_id,item_cnt_day
20602,20949,187642.0
2749,2808,17245.0
3654,3732,16642.0
17418,17717,15830.0
5717,5822,14515.0
3656,3734,11688.0
6543,6675,10289.0
3653,3731,10099.0
1814,1855,10032.0
16493,16787,9227.0


In [ ]:
#2
sales.loc[(sales.date_block_num < 36, 'year')] = 2015
sales.loc[(sales.date_block_num < 24, 'year')] = 2014
sales.loc[(sales.date_block_num < 12, 'year')] = 2013
sales.drop_duplicates(subset=['item_id', 'year'], inplace=True)
years = sales.groupby('item_id', as_index=False)['year'].sum()
years = years.loc[(years.year == 6042)]
newresult = result.head(10).merge(years)
newresult

,item_id,item_cnt_day,year
0,20949,187642.0,6042.0
1,2808,17245.0,6042.0
2,3732,16642.0,6042.0
3,17717,15830.0,6042.0
4,5822,14515.0,6042.0
5,3734,11688.0,6042.0
6,6675,10289.0,6042.0
7,1855,10032.0,6042.0
8,16787,9227.0,6042.0


# Задание 2

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ВиМ/Лаба 1/items.csv")
items_frame = pd.DataFrame(data)
data = pd.read_csv("/content/drive/MyDrive/ВиМ/Лаба 1/item_categories.csv")
categories_frame = pd.DataFrame(data)
items_copy = items_frame.copy()
items_frame = items_frame.join(result.set_index('item_id'))
items_frame = items_frame.groupby('category_id', as_index=False).sum()
items_frame = items_frame.sort_values(by='item_cnt_day', ascending=False)
print(f"Названия и Id для минимальных:\n{categories_frame.loc[items_frame['item_cnt_day'] == items_frame.item_cnt_day.min()]}")

Названия и Id для минимальных:
               item_category_name  item_category_id
10            Game consoles - PS2                10
51   Books - Cognitive literature                51


In [ ]:
print(f"Название и Id для максимального:\n{categories_frame.loc[items_frame['item_cnt_day'] == items_frame.item_cnt_day.max()]}")

Название и Id для максимального:
   item_category_name  item_category_id
40       Cinema - DVD                40


# Задание 3

1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.
2. Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.


In [ ]:
#3.1
data = pd.read_csv("/content/drive/MyDrive/ВиМ/Лаба 1/sales_train.csv")
sales_train = pd.DataFrame(data)
sales_frame = pd.DataFrame(data)
data = pd.read_csv("/content/drive/MyDrive/ВиМ/Лаба 1/items.csv")
items_train = pd.DataFrame(data)
sales_train = sales_train.join(items_train.set_index('item_id'), on='item_id')
shop_sales = sales_frame.groupby('shop_id', as_index=False)['item_cnt_day'].sum()
shop_sales = shop_sales.sort_values(by='item_cnt_day', ascending=False)
shop_sales = shop_sales.head(5)
min_category1 = sales_train[sales_train['category_id'] == 51]
min_category2 = sales_train[sales_train['category_id'] == 10]
min_category = min_category1.append(min_category2)
min_category = min_category[min_category['shop_id'].isin(shop_sales['shop_id'])]
min_category = min_category.groupby('item_id', as_index=False)['item_cnt_day'].sum()
min_category
#продаж для минимальной категории нет в топовых магазинах

,item_id,item_cnt_day


In [ ]:
max_category = sales_train[sales_train['category_id'] == 40]
max_category = max_category[max_category['shop_id'].isin(shop_sales['shop_id'])]
max_category = max_category.groupby(['item_id', 'shop_id'], as_index=False)['item_cnt_day'].sum()
max_category = max_category.groupby('item_id', as_index=False)['item_cnt_day'].mean()
max_category.sort_values(by='item_id')

,item_id,item_cnt_day
0,0,1.00
1,2,2.00
2,3,2.00
3,4,1.00
4,5,1.00
...,...,...
4709,22157,1.00
4710,22159,2.00
4711,22160,4.50
4712,22162,55.40


In [ ]:
#3.2
max_months = sales_train[sales_train['item_id'].isin(max_category['item_id'])]
max_months = max_months.append(sales_train[sales_train['item_id'].isin(min_category['item_id'])])
max_months = max_months.groupby(['date_block_num', 'item_id'], as_index=False)['item_cnt_day'].sum()
min_months = max_months.drop_duplicates('item_id', keep='last')
max_months = max_months.drop_duplicates('item_id', keep='first')
max_months.sort_values(by='item_id')

,date_block_num,item_id,item_cnt_day
39794,20,0,1.0
38490,19,2,1.0
37044,18,3,1.0
39795,20,4,1.0
43140,23,5,1.0
...,...,...,...
4693,1,22157,1.0
49184,28,22159,12.0
2333,0,22160,11.0
38489,18,22162,1.0


In [ ]:
min_months.sort_values(by='item_id')

,date_block_num,item_id,item_cnt_day
39794,20,0,1.0
42025,22,2,1.0
38491,19,3,1.0
39795,20,4,1.0
43140,23,5,1.0
...,...,...,...
21122,8,22157,1.0
50033,29,22159,3.0
37043,17,22160,1.0
53277,33,22162,10.0


# Задание 4

1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
#4.1
sales_train_c = sales_train.copy()
jeneral_category = min_category.append(max_category)
sales_train_c = sales_train[sales_train['item_id'].isin(jeneral_category['item_id'])]
sales_train_median = sales_train_c.groupby(['date_block_num', 'category_id'], as_index=False)['item_price'].median()
sales_train_median.sort_values(by='date_block_num')

,date_block_num,category_id,item_price
0,0,40,199.0
1,1,40,203.0
2,2,40,231.0
3,3,40,199.0
4,4,40,199.0
5,5,40,199.0
6,6,40,149.0
7,7,40,149.0
8,8,40,198.0
9,9,40,199.0


In [ ]:
sales_train_mean = sales_train_c.groupby(['date_block_num', 'category_id'], as_index=False)['item_price'].mean()
sales_train_mean.sort_values(by='date_block_num')

,date_block_num,category_id,item_price
0,0,40,244.858909
1,1,40,252.399737
2,2,40,260.897975
3,3,40,254.973153
4,4,40,247.721981
5,5,40,245.963749
6,6,40,236.863751
7,7,40,234.487469
8,8,40,241.838343
9,9,40,247.574230


In [ ]:
#4.2
year2013 = sales_train_mean.loc[sales_train_mean.date_block_num < 12]
for row in range(1, len(year2013) - 1):
  if ((year2013.iloc[row].item_price < year2013.iloc[row-1].item_price) and (year2013.iloc[row].item_price < year2013.iloc[row+1].item_price)):
    display(year2013.loc[row])

date_block_num      7.000000
category_id        40.000000
item_price        234.488519
Name: 8, dtype: float64

In [ ]:
year2014 = sales_train_mean.loc[sales_train_mean.date_block_num < 24]
year2014 = year2014.loc[sales_train_mean.date_block_num >= 12]
for row in range(1, len(year2014) - 1):
  if ((year2014.iloc[row].item_price < year2014.iloc[row-1].item_price) and (year2014.iloc[row].item_price < year2014.iloc[row+1].item_price)):
    display(year2014.iloc[row])

date_block_num     16.000000
category_id        40.000000
item_price        254.372031
Name: 17, dtype: float64

date_block_num     18.0
category_id        51.0
item_price        129.0
Name: 20, dtype: float64

date_block_num     21.000000
category_id        40.000000
item_price        263.760901
Name: 23, dtype: float64

In [ ]:
year2015 = sales_train_mean.loc[sales_train_mean.date_block_num < 36]
year2015 = year2015.loc[sales_train_mean.date_block_num >= 24]
for row in range(1, len(year2015) - 1):
  if ((year2015.iloc[row].item_price < year2015.iloc[row-1].item_price) and (year2015.iloc[row].item_price < year2015.iloc[row+1].item_price)):
    display(year2015.iloc[row])

date_block_num     25.000000
category_id        40.000000
item_price        256.363506
Name: 27, dtype: float64

date_block_num     31.000000
category_id        40.000000
item_price        273.180294
Name: 33, dtype: float64